<a href="https://colab.research.google.com/github/KostaKat/MAT442/blob/main/hw3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Logistic Regression

Logistic regression is a model that uses a **logistic function** to perform binary classification, and can also be extended to multi-class classification.


Given samples $ \{(\alpha_i, b_i) : i = 1, \dots, n\} $ , where:
- $ \alpha_i \in \mathbb{R}^d $ are the **features** (a vector),
- $ b_i \in \{0, 1\} $ is the **label** (binary).

Given the input samples we define a matrix $ A \in \mathbb{R}^{n \times d} $ with rows $ \alpha_i^T $ for $ i = 1, \dots, n $, and a label vector $ b = (b_1, \dots, b_n)^T \in \{0, 1\}^n $.

The goal of logistic regression is to **approximate the probability** of a sample having a label of 1. This is done through the **logit function**, which expresses the relationship between the features of a sample and the probability of the label being 1 as a linear function. This is defined to be a regression problem and is formally defined as:
$$
p(\alpha;x) = \log \left( \frac{p(\alpha ; x)}{1 - p(\alpha ; x)} \right) = \alpha^T x
$$
Where $ x \in \mathbb{R}^d $ are the model parameters.


Thus, the probability $ p(\alpha) $ is given by:
$$
p(\alpha; x) = \sigma(\alpha^T x) = \frac{1}{1 + e^{-\alpha^T x}}
$$




To estimate the model parameters, we maximize the **likelihood** of the data:
$$
\mathcal{L}(x; A, b) = \prod_{i=1}^n p(\alpha_i ; x)^{b_i} (1 - p(\alpha_i ; x))^{1 - b_i}
$$
To make the process more efficient, applications use the negative log-likelihood (also known as the **cross-entropy loss**) is:
$$
\ell(x; A, b) = - \frac{1}{n} \sum_{i=1}^n b_i \log(\sigma(\alpha^T x)) - \frac{1}{n} \sum_{i=1}^n (1 - b_i) \log(1 - \sigma(\alpha^T x))
$$


Ultimately making it a minimization problem, that minimizes the cross-entropy loss.

It is apparent that various approaches can be taken. For example, taking the gradient of the loss function, which will give us the global minimum. However, as discussed in the previous section, this approach is inefficient, especially for large datasets. Thus,  we use **stochastic gradient descent**, which is formally expressed as:
$$
x^{k+1} = x^k + \beta \left( b_i - \sigma(\alpha_i^T x^k) \right) \alpha_i
$$
where $ i $ is selected randomly from $ \{1, \dots, n\} $ and $ \beta $ is the step size.



In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Cross-entropy loss
def compute_loss(A, b, x):
    n = len(b)
    predictions = sigmoid(np.dot(A, x))
    loss = - (1 / n) * (np.dot(b, np.log(predictions)) + np.dot((1 - b), np.log(1 - predictions)))
    return loss

# Stochastic Gradient Descent (SGD)
def stochastic_gradient_descent(A, b, x_init, learning_rate, epochs):
    n, d = A.shape
    x = x_init.copy()

    for epoch in range(epochs):
        # Pick a random sample
        i = np.random.randint(n)

        # Compute gradient for the selected sample
        gradient = (b[i] - sigmoid(np.dot(A[i], x))) * A[i]

        # Update rule for stochastic gradient descent
        x = x + learning_rate * gradient

        # print loss every 10 epochs
        if epoch % 10 == 0:
            loss = compute_loss(A, b, x)
            print(f"Epoch {epoch}, Loss: {loss}")

    return x

# Prediction function
def predict(A, x):
    probabilities = sigmoid(np.dot(A, x))
    return (probabilities >= 0.5).astype(int)

# Load the Iris dataset and use only two classes
iris = datasets.load_iris()
A = iris.data[iris.target != 2]  # Use only setosa and versicolor
b = iris.target[iris.target != 2]  # Setosa is class 0 and versicolor is class 1

# Split the dataset into training and test sets
A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=42)

# Initialize parameters for logistic regression
x_init = np.zeros(A_train.shape[1])

# train model
learning_rate = 0.01
epochs = 100
learned_x = stochastic_gradient_descent(A_train, b_train, x_init, learning_rate, epochs)

# test model
b_pred = predict(A_test, learned_x)

# Calculate accuracy
accuracy = accuracy_score(b_test, b_pred)
print(f"\nAccuracy on the test set: {accuracy * 100:.2f}%")


Epoch 0, Loss: 0.6744941702980122
Epoch 10, Loss: 0.661699873896178
Epoch 20, Loss: 0.6793574595548757
Epoch 30, Loss: 0.5611247510844308
Epoch 40, Loss: 0.5622386349398653
Epoch 50, Loss: 0.5021251717517545
Epoch 60, Loss: 0.49621516913357183
Epoch 70, Loss: 0.4608039882151945
Epoch 80, Loss: 0.46508414665590614
Epoch 90, Loss: 0.4115264397943958

Accuracy on the test set: 100.00%
